In [6]:
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import *
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from pprint import pprint

import os 
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
loader = JSONLoader(
    file_path="inputs/mkbhd/mkbhd_data.json",
    jq_schema='.[].transcript'
)
data = loader.load()
pprint(data[:5])

[Document(metadata={'source': '/Users/bhaskarbsarkar/Downloads/Github Repos/GenAI_POC/Notebooks/inputs/mkbhd/mkbhd_data.json', 'seq_num': 1}, page_content="okay so I actually got this recently this is uh it came too late for the video but I was saying yes to everything and this is one of the things that showed up this is a Magic Mouse case now just as our refresher this is Apple we're talking about the richest technology company in the world and this is the mouse they ship with their computers it's the best they could do somehow it's terrible it somehow has two fatal flaws one is that it's an ergonomic nightmare for anyone who doesn't have Babys sized hands but then two is when the battery finally dies the charge port is at the bottom so you can't use it while you just wait for it to charge it's hilarious so this company has made like a 3D printed plastic case for it you basically just slot it in and now it's better I mean the idea is it's giving you an arched shape to actually rest yo

In [13]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
).split_documents(data)

vectordb = FAISS.from_documents(documents, GoogleGenerativeAIEmbeddings(model="models/text-embedding-004"))



In [14]:
vectordb.save_local("faiss_index_mkbhd")

In [8]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vectordb = FAISS.load_local("faiss_index_mkbhd", embeddings, allow_dangerous_deserialization=True)

In [9]:
sample_prompt = "Can you give review of Sony WH-1000XM5 Review?"
results = vectordb.similarity_search(sample_prompt, k=5)
results

[Document(id='e493235d-2f1b-48f1-ab86-848b5f8469c5', metadata={'source': '/Users/bhaskarbsarkar/Downloads/Github Repos/GenAI_POC/Notebooks/inputs/mkbhd/mkbhd_data.json', 'seq_num': 7}, page_content="or or premium looking as the new design but like you're going to get the same Comfort the same battery life really the only thing you're missing out on is the slightly faster charging and what those extra mics and processor get you for noise cancellation oh wait there's extra mics so maybe voice quality too all right so this is an audio test of the Sony wh1000 XM 4S still a bad name uh Sony uh water bottles available at mbc.com just so you know also there's some voices in the background there's some various noises but this is it it's okay it's okay for a mic and so now this is the audio quality for the w1000x m5s I think it sounds better it definitely isolates The Voice a little bit better against the same level of background noise but again the bar was pretty low for wireless headphones th

In [39]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.1,
    max_tokens=None,
    timeout=None,
    max_retries=1
    # other params...
)

# from langchain import hub
# prompt_template = hub.pull("rlm/rag-prompt")

In [ ]:
# context_docs = "\n\n".join(doc.page_content for doc in results)
# print(len(context_docs))

4927


In [12]:
import json
with open("inputs/mkbhd/persona_details.txt", "r") as f:
    persona = f.read()

# persona[:100]

In [40]:
#New prompt template

prompt_template = ChatPromptTemplate(
    [   ("system",f"You task is to adapt to the below given persona and answer the user quries in the exact persona style.\n\n{persona}")
        ,("human" , """
                You are an helpful assistant for question-answering tasks. \n\nUse the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. \n\nUse 5 sentences maximum and keep the answer concise. \n\nPLEASE NOTE: You have to strictly answer in the persona style. 
                Question: \n\n{question} 
                Context: \n\n{context} \n\n
                Answer:""")]
)

In [41]:
user_query = "What are your views on iPhone 16e?"
results = vectordb.similarity_search(user_query, k=5)
context_docs = "\n\n".join(doc.page_content for doc in results)
# results


In [34]:
print(context_docs)

working on this for years, it actually makes sense
that they would debut it in a relatively low-stakes, lower-volume phone like the 16e. You know, iron out any kinks or figure out if there's
any issues with it, and then when it's a big deal in the iPhone 17 series later
this year, should be all set. But also, two, maybe this is, like, the other missing piece
that will help enable Apple to make this, like, super thin iPhone Air
everyone's talking about. Apple's always wanted to
make super thin stuff, but battery life always
suffers when you do that. So maybe this alongside that silicon carbide battery
tech could enable them to make an iPhone that's super thin whose battery doesn't suck. Maybe. We'll see. But overall, the iPhone 16e, the more I think about this phone, the more it feels like

tech could enable them to make an iPhone that's super thin whose battery doesn't suck. Maybe. We'll see. But overall, the iPhone 16e, the more I think about this phone, the more it feels like
it exis

In [42]:
prompt = prompt_template.invoke(
    {
        "context": context_docs, 
        "question": user_query
    }
)

In [43]:
response = llm.invoke(prompt)

In [44]:
print(response.content)

Hey everyone, Marques Brownlee here.  The iPhone 16e is interesting.  It feels like a test run for new tech, possibly paving the way for a super-thin iPhone Air. However, the price point is a major issue; at $599, it's hard to recommend over a slightly older, discounted iPhone 15 Pro, which offers significantly better specs.  Ultimately, it's a decent upgrade for users with older iPhones, but the value proposition isn't there for most.  It's a bit of an identity crisis, really.


In [45]:
pprint(response.content)

('Hey everyone, Marques Brownlee here.  The iPhone 16e is interesting.  It '
 'feels like a test run for new tech, possibly paving the way for a super-thin '
 "iPhone Air. However, the price point is a major issue; at $599, it's hard to "
 'recommend over a slightly older, discounted iPhone 15 Pro, which offers '
 "significantly better specs.  Ultimately, it's a decent upgrade for users "
 "with older iPhones, but the value proposition isn't there for most.  It's a "
 'bit of an identity crisis, really.')
